# Decision Trees Using Gradient Boosting Algorithm

## Import Nescessary Packages

In [1]:
import numpy

## Class Definition

In [11]:
class DecisionTree():
  #Initializing the Decision Tree with hyperparameters
  """
    max_depth: Maximum depth of the tree
    min_sample_leaf: Minimum samples required in a leaf node
    min_information_gain: Minimum information gain for a split to be considered
    loss: Loss function to optimize. Default is square_error
  """
  def __init__(self, max_depth=5, min_sample_leaf=5, min_information_gain=0.0, loss="square_error") -> None:
        self.max_depth = max_depth
        self.min_sample_leaf = min_sample_leaf
        self.min_information_gain = min_information_gain
        self.loss = loss

  #Computing the entropy value of class probabilities (class_prob)
  def entropy(self, class_prob):
        return numpy.sum([-p * numpy.log(p) for p in class_prob if p > 0])

  #Calculating entropy for the given dataset's labels
  def entropy_data(self, labels):
        unique, counts = numpy.unique(labels, return_counts=True)
        probs_each_label = counts / labels.shape[0]
        return self.entropy(probs_each_label)

  #Computing the weighted average entropy of partitions(subsets)
  def partition_entropy(self, subsets):
        count = numpy.sum([subset.shape[0] for subset in subsets])
        return numpy.sum([self.entropy_data(subset[:, -1]) * subset.shape[0] for subset in subsets]) / count

  #Spliting the dataset(data) based on a feature (f_id) and its value (f_value)
  def feature_split(self, data, f_id, f_value):
        mask = data[:, f_id] < f_value
        g1 = data[mask]
        g2 = data[~mask]
        return g1, g2
  #Finding the best feature and value & best and minimum partition to split the data on
  def get_best_split(self, data):
        min_partition_entropy = float('inf')
        min_entropy_feature = None
        min_entropy_value = None
        min_g1 = None
        min_g2 = None

        for id in range(data.shape[1] - 1):
            unique_feature_values = numpy.unique(data[:, id])
            for value in unique_feature_values:
                g1, g2 = self.feature_split(data, id, value)
                if len(g1) < self.min_sample_leaf or len(g2) < self.min_sample_leaf:
                    continue
                partition_entropy = self.partition_entropy([g1, g2])
                if partition_entropy < min_partition_entropy:
                    min_partition_entropy = partition_entropy
                    min_entropy_feature = id
                    min_entropy_value = value
                    min_g1 = g1
                    min_g2 = g2

        return min_g1, min_g2, min_entropy_feature, min_entropy_value, min_partition_entropy

  #Creating a leaf node for the tree
  def leaf_node(self, data, labels):
        if self.loss == "square_error":
            prediction = numpy.mean(labels)
        else:
            prediction = numpy.mean(labels)
        return TreeNode(data, None, None, prediction, 0)

  #Recursively building the decision tree and returns the root of the tree
  def build_tree(self, data, current_depth):
        labels = data[:, -1]
        if (current_depth >= self.max_depth) or (len(numpy.unique(labels)) == 1):
            return self.leaf_node(data, labels)

        g1, g2, feature, value, information_gain = self.get_best_split(data)

        if feature is None or information_gain < self.min_information_gain:
            return self.leaf_node(data, labels)

        left = self.build_tree(g1, current_depth + 1)
        right = self.build_tree(g2, current_depth + 1)

        prediction = numpy.mean(labels)
        new_node = TreeNode(data=data, feature_id=feature, feature_value=value, prediction=prediction, information_gain=information_gain)
        new_node.left = left
        new_node.right = right
        return new_node

  #Fitting the decision tree model to the data
  def fit(self, data):
        tree_root = self.build_tree(data, 0)
        return DecisionTreeResults(tree_root=tree_root)

class TreeNode():
  #Representing a single node in the decision tree
  ''' data: Data at the node
      feature_id: Feature index used for splitting
      feature_value: Feature value to split on
      prediction: Prediction value at the node
      information_gain: Information gain for the split
  '''
  def __init__(self, data, feature_id, feature_value, prediction, information_gain) -> None:
        self.data = data
        self.feature_id = feature_id
        self.feature_value = feature_value
        self.prediction = prediction
        self.information_gain = information_gain
        self.left = None
        self.right = None

class DecisionTreeResults():
  #Wrapping the trained decision tree for making predictions
  def __init__(self, tree_root) -> None:
        self.tree_root = tree_root

  #Predicting the output for a single data row
  def predict_single_row(self, node, row):
        if node.feature_id is None:
            return node.prediction
        if row[node.feature_id] < node.feature_value:
            return self.predict_single_row(node.left, row)
        else:
            return self.predict_single_row(node.right, row)

  #Predicting the outputs for the entire dataset
  def predict(self, data):
        result = []
        for row in data:
            result.append(self.predict_single_row(self.tree_root, row))
        return numpy.array(result)

In [12]:
#Computing the sigmoid activation function.
def sigmoid(x):
    return 1 / (1 + numpy.exp(-x))

#Computing the softmax activation function along a specified axis.
def softmax(x, axis=1):
    shiftx = x - numpy.max(x, axis=axis, keepdims=True)
    exps = numpy.exp(shiftx)
    return exps / numpy.sum(exps, axis=axis, keepdims=True)

In [13]:
class GradientBoostedTree():
# Initializing parameters for the gradient boosted tree model
        # B: Number of boosting rounds (trees to build)
        # lmd: Learning rate to scale contributions of each tree
        # max_depth: Maximum depth of each decision tree
        # min_sample_leaf: Minimum number of samples per leaf node
        # min_information_gain: Minimum information gain to split a node
        # loss: Loss function used for optimization ("square_error" or classification)
    def __init__(self, B=100, lmd=0.1, max_depth=3, min_sample_leaf=5, min_information_gain=0.0, loss="square_error"):
        self.B = B
        self.lmd = lmd
        self.max_depth = max_depth
        self.min_sample_leaf = min_sample_leaf
        self.min_information_gain = min_information_gain
        self.loss = loss
        self.tree_list = []
        self.n_classes = None

#Training the gradient boosted tree model on the provided dataset
    def fit(self, data):
        X, y = data[:, :-1], data[:, -1]
        self.n_classes = len(numpy.unique(y))

    #Handling regression (square error loss)
        if self.loss == "square_error":
            self.initial_predictions = numpy.full(X.shape[0], numpy.mean(y))
            current_predictions = self.initial_predictions.copy()

            for i in range(self.B):
                #Computing residuals
                residuals = y - current_predictions
                tree = DecisionTree(self.max_depth, self.min_sample_leaf, self.min_information_gain, "square_error")
                #Combining features with residuals as target
                tree_data = numpy.column_stack((X, residuals))
                #Training decision tree on residuals
                fitted_tree = tree.fit(tree_data)
                #Storing the tree
                self.tree_list.append(fitted_tree)
                #Updating predictions
                current_predictions += self.lmd * fitted_tree.predict(X)

    #Handling binary classification
        elif self.n_classes == 2:
            self.initial_predictions = numpy.zeros(X.shape[0]) # Starting with all-zero predictions
            current_predictions = self.initial_predictions.copy()

            for i in range(self.B):
                #Calculating probabilities using sigmoid function.
                p = sigmoid(current_predictions)
                residuals = y - p
                tree = DecisionTree(self.max_depth, self.min_sample_leaf, self.min_information_gain, "square_error")
                tree_data = numpy.column_stack((X, residuals))
                fitted_tree = tree.fit(tree_data)
                self.tree_list.append(fitted_tree)
                current_predictions += self.lmd * fitted_tree.predict(X)

    #Handling multi-class classification
        else:
            self.initial_predictions = numpy.zeros((X.shape[0], self.n_classes))
            current_predictions = self.initial_predictions.copy()

            for i in range(self.B):
                gradients = self.calculate_residuals(y, current_predictions)
                trees_per_class = []
                for k in range(self.n_classes):
                    tree = DecisionTree(self.max_depth, self.min_sample_leaf, self.min_information_gain, "square_error")
                    tree_data = numpy.column_stack((X, gradients[:, k]))
                    fitted_tree = tree.fit(tree_data)
                    trees_per_class.append(fitted_tree)
                    current_predictions[:, k] += self.lmd * fitted_tree.predict(X) # Update class-specific predictions

                self.tree_list.append(trees_per_class)
    #Returning the trained model
        return GradientBoostedTreeResults(self.tree_list, self.initial_predictions, self.lmd, self.loss, self.n_classes)

#Calculating residuals for multi-class classification using the softmax function
    def calculate_residuals(self, y, current_predictions):
        p = softmax(current_predictions)
        y_one_hot = numpy.zeros((len(y), self.n_classes)) # One-hot encode labels
        y_one_hot[numpy.arange(len(y)), y.astype(int)] = 1
        return y_one_hot - p # Residuals = true labels - predicted probabilities

class GradientBoostedTreeResults():
    def __init__(self, trees, initial_predictions, lmd, loss, n):
        self.trees = trees
        self.initial_predictions = initial_predictions
        self.lmd = lmd
        self.loss = loss
        self.n = n

 #Predicting outputs for new input data (X) using the trained model
    def predict(self, X):
        if self.loss == "square_error":
            predictions = numpy.copy(self.initial_predictions)
            for tree in self.trees:
                #Aggregating predictions from all trees
                predictions += self.lmd * tree.predict(X)
            return predictions

        #Binary classification predictions
        elif self.n == 2:
            predictions = numpy.copy(self.initial_predictions)
            for tree in self.trees:
                predictions += self.lmd * tree.predict(X)
            probabilities = sigmoid(predictions)
            #Converting probabilities to binary labels
            return (probabilities >= 0.5).astype(int)

        #Multi-class classification predictions
        else:
            predictions = numpy.copy(self.initial_predictions)
            for trees_per_class in self.trees:
                for k, tree in enumerate(trees_per_class):
                    predictions[:, k] += self.lmd * tree.predict(X)
            probabilities = softmax(predictions)
            #Returning the class with the highest probability.
            return numpy.argmax(probabilities, axis=1)

## Testing the Model

### The Iris DataSet Test

In [ ]:
from sklearn.datasets import load_iris

In [ ]:
#Loading the Iris dataset using the `load_iris` function from sklearn.datasets
iris = load_iris()
#Extracting the feature matrix (X) containing the measurements for the Iris dataset
X = iris.data
#Extracting the target array (y) containing the class labels
y = iris.target
#Combining the (X) and (y) into a single dataset where the last column represents the target labels.
data = numpy.column_stack((X, y))

In [ ]:
# Extract the target array (y) containing the class labels for the Iris dataset.# tree = DecisionTree(max_depth=4, min_sample_leaf=1, min_information_gain=0.01, loss="logistic")
# tree_results = tree.fit(data)

In [ ]:
tree = GradientBoostedTree(lmd=0.1,max_depth=4, min_sample_leaf=1, min_information_gain=0.01, loss="logistic")
tree_results = tree.fit(data)

In [ ]:
len(tree_results.trees)

100

In [ ]:
preds = tree_results.predict(X)

In [ ]:
print("Predicted vs True labels:")
for pred, true in zip(preds, y):
    print(f"Predicted: {pred}, True: {true}")

Predicted vs True labels:
Predicted: 0, True: 0
Predicted: 0, True: 0
Predicted: 0, True: 0
Predicted: 0, True: 0
Predicted: 0, True: 0
Predicted: 0, True: 0
Predicted: 0, True: 0
Predicted: 0, True: 0
Predicted: 0, True: 0
Predicted: 0, True: 0
Predicted: 0, True: 0
Predicted: 0, True: 0
Predicted: 0, True: 0
Predicted: 0, True: 0
Predicted: 0, True: 0
Predicted: 0, True: 0
Predicted: 0, True: 0
Predicted: 0, True: 0
Predicted: 0, True: 0
Predicted: 0, True: 0
Predicted: 0, True: 0
Predicted: 0, True: 0
Predicted: 0, True: 0
Predicted: 0, True: 0
Predicted: 0, True: 0
Predicted: 0, True: 0
Predicted: 0, True: 0
Predicted: 0, True: 0
Predicted: 0, True: 0
Predicted: 0, True: 0
Predicted: 0, True: 0
Predicted: 0, True: 0
Predicted: 0, True: 0
Predicted: 0, True: 0
Predicted: 0, True: 0
Predicted: 0, True: 0
Predicted: 0, True: 0
Predicted: 0, True: 0
Predicted: 0, True: 0
Predicted: 0, True: 0
Predicted: 0, True: 0
Predicted: 0, True: 0
Predicted: 0, True: 0
Predicted: 0, True: 0
Predic

In [ ]:
accuracy = numpy.sum(preds == y) / len(y)
print(f"Accuracy: {accuracy * 100:.2f}%")

Accuracy: 98.00%


### The Wine DataSet Test

In [ ]:
from sklearn.datasets import load_wine

In [ ]:
wine = load_wine()
X = wine.data
y = wine.target

In [ ]:
data = numpy.column_stack((X, y))
#tree = DecisionTree(max_depth=4, min_sample_leaf=1, min_information_gain=0.01, loss="logistic")
tree = GradientBoostedTree(lmd=0.1,max_depth=4, min_sample_leaf=1, min_information_gain=0.01, loss="logistic")
tree_results = tree.fit(data)
preds = tree_results.predict(X)
print("Predicted vs True labels:")
for pred, true in zip(preds, y):
    print(f"Predicted: {pred}, True: {true}")

Predicted vs True labels:
Predicted: 0, True: 0
Predicted: 0, True: 0
Predicted: 0, True: 0
Predicted: 0, True: 0
Predicted: 1, True: 0
Predicted: 0, True: 0
Predicted: 0, True: 0
Predicted: 0, True: 0
Predicted: 0, True: 0
Predicted: 0, True: 0
Predicted: 0, True: 0
Predicted: 0, True: 0
Predicted: 0, True: 0
Predicted: 0, True: 0
Predicted: 0, True: 0
Predicted: 0, True: 0
Predicted: 0, True: 0
Predicted: 0, True: 0
Predicted: 0, True: 0
Predicted: 0, True: 0
Predicted: 0, True: 0
Predicted: 0, True: 0
Predicted: 0, True: 0
Predicted: 0, True: 0
Predicted: 0, True: 0
Predicted: 0, True: 0
Predicted: 0, True: 0
Predicted: 0, True: 0
Predicted: 0, True: 0
Predicted: 0, True: 0
Predicted: 0, True: 0
Predicted: 0, True: 0
Predicted: 0, True: 0
Predicted: 0, True: 0
Predicted: 0, True: 0
Predicted: 0, True: 0
Predicted: 0, True: 0
Predicted: 0, True: 0
Predicted: 0, True: 0
Predicted: 0, True: 0
Predicted: 0, True: 0
Predicted: 0, True: 0
Predicted: 0, True: 0
Predicted: 1, True: 0
Predic

In [ ]:
accuracy = numpy.sum(preds == y) / len(y)
print(f"Accuracy: {accuracy * 100:.2f}%")

Accuracy: 97.75%


### Breat Cancer DataSet

In [ ]:
from sklearn.datasets import load_breast_cancer

In [ ]:
breast_cancer = load_breast_cancer()
X = breast_cancer.data
y = breast_cancer.target

In [ ]:
data = numpy.column_stack((X, y))
# tree = DecisionTree(max_depth=10, min_sample_leaf=1, min_information_gain=0.01, loss="logistic")
tree = GradientBoostedTree(lmd=0.1,max_depth=4, min_sample_leaf=1, min_information_gain=0.01, loss="logistic")
tree_results = tree.fit(data)
preds = tree_results.predict(X)
print("Predicted vs True labels:")
for pred, true in zip(preds, y):
    print(f"Predicted: {pred}, True: {true}")

Predicted vs True labels:
Predicted: 0, True: 0
Predicted: 0, True: 0
Predicted: 0, True: 0
Predicted: 0, True: 0
Predicted: 0, True: 0
Predicted: 0, True: 0
Predicted: 0, True: 0
Predicted: 0, True: 0
Predicted: 0, True: 0
Predicted: 0, True: 0
Predicted: 0, True: 0
Predicted: 0, True: 0
Predicted: 0, True: 0
Predicted: 0, True: 0
Predicted: 0, True: 0
Predicted: 0, True: 0
Predicted: 0, True: 0
Predicted: 0, True: 0
Predicted: 0, True: 0
Predicted: 1, True: 1
Predicted: 1, True: 1
Predicted: 1, True: 1
Predicted: 0, True: 0
Predicted: 0, True: 0
Predicted: 0, True: 0
Predicted: 0, True: 0
Predicted: 0, True: 0
Predicted: 0, True: 0
Predicted: 0, True: 0
Predicted: 0, True: 0
Predicted: 0, True: 0
Predicted: 0, True: 0
Predicted: 0, True: 0
Predicted: 0, True: 0
Predicted: 0, True: 0
Predicted: 0, True: 0
Predicted: 0, True: 0
Predicted: 1, True: 1
Predicted: 1, True: 0
Predicted: 0, True: 0
Predicted: 1, True: 0
Predicted: 0, True: 0
Predicted: 0, True: 0
Predicted: 0, True: 0
Predic

In [ ]:
accuracy = numpy.sum(preds == y) / len(y)
print(f"Accuracy: {accuracy * 100:.2f}%")

Accuracy: 97.54%


### Diabetes Housing DataSet

In [ ]:
from sklearn.datasets import load_diabetes
diabetes = load_diabetes()
X, y = diabetes.data, diabetes.target

In [ ]:
data = numpy.column_stack((X, y))
# tree = DecisionTree(max_depth=20, min_sample_leaf=1, min_information_gain=0.01, loss="logistic")
tree = GradientBoostedTree(B=50,lmd=0.1,max_depth=4, min_sample_leaf=1, min_information_gain=0.01)
tree_results = tree.fit(data)
preds = tree_results.predict(X)
print("Predicted vs True labels:")
for pred, true in zip(preds, y):
    print(f"Predicted: {pred}, True: {true}")

Predicted vs True labels:
Predicted: 182.5321755358343, True: 151.0
Predicted: 89.16620038946364, True: 75.0
Predicted: 182.5321755358343, True: 141.0
Predicted: 184.67353238266574, True: 206.0
Predicted: 98.84747536558898, True: 135.0
Predicted: 84.38323647517007, True: 97.0
Predicted: 89.16620038946364, True: 138.0
Predicted: 179.7850072671412, True: 63.0
Predicted: 182.5321755358343, True: 110.0
Predicted: 187.09162751278382, True: 310.0
Predicted: 89.16620038946364, True: 101.0
Predicted: 204.00713197446248, True: 69.0
Predicted: 106.19630968087299, True: 179.0
Predicted: 182.5321755358343, True: 185.0
Predicted: 106.19630968087299, True: 118.0
Predicted: 184.67353238266574, True: 171.0
Predicted: 143.1664534712103, True: 166.0
Predicted: 179.7850072671412, True: 144.0
Predicted: 133.863876003072, True: 97.0
Predicted: 130.44568641019458, True: 168.0
Predicted: 96.47751350961569, True: 68.0
Predicted: 106.19630968087299, True: 49.0
Predicted: 167.6446448322342, True: 68.0
Predicted